In [1]:
"""
This example loads the pre-trained bert-base-nli-mean-tokens models from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime

In [2]:
pwd

'/home/vv/git/chirag'

In [3]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [4]:
# Read the dataset
model_name = 'roberta-base-nli-stsb-mean-tokens'
train_batch_size = 8
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
sts_reader = STSDataReader('./stsbenchmark', normalize_scores=True)

In [5]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_data = SentencesDataset(sts_reader.get_examples('sts-train.csv'), model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('sts-dev.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)


2019-12-17 05:48:22 - Load pretrained SentenceTransformer: roberta-base-nli-stsb-mean-tokens
2019-12-17 05:48:22 - Did not find a / or \ in the name. Assume to download model from server
2019-12-17 05:48:22 - Load SentenceTransformer from folder: /home/vv/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip
2019-12-17 05:48:22 - loading configuration file /home/vv/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip/0_RoBERTa/config.json
2019-12-17 05:48:22 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers":

Convert dataset:   7%|▋         | 329/5051 [00:00<00:01, 3282.01it/s]

2019-12-17 05:48:28 - Read STSbenchmark train dataset


Convert dataset:  35%|███▌      | 531/1500 [00:00<00:00, 5305.79it/s]

2019-12-17 05:48:29 - Num sentences: 5051
2019-12-17 05:48:29 - Sentences 0 longer than max_seqence_length: 0
2019-12-17 05:48:29 - Sentences 1 longer than max_seqence_length: 0
2019-12-17 05:48:29 - Read STSbenchmark dev dataset


Convert dataset: 100%|██████████| 1500/1500 [00:00<00:00, 3183.25it/s]

2019-12-17 05:48:30 - Num sentences: 1500
2019-12-17 05:48:30 - Sentences 0 longer than max_seqence_length: 0
2019-12-17 05:48:30 - Sentences 1 longer than max_seqence_length: 0


In [6]:
# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_data)*num_epochs/train_batch_size*0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2019-12-17 05:48:30 - Warmup-steps: 253


In [7]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Iteration:  20%|██        | 128/632 [00:17<01:09,  7.26it/s]


Iteration:  41%|████      | 256/632 [00:34<00:55,  6.83it/s]


Iteration:  61%|██████    | 384/632 [00:52<00:33,  7.45it/s]


Iteration:  81%|████████  | 512/632 [01:10<00:16,  7.24it/s]


Convert Evaluating:   3%|▎         | 5/188 [00:00<00:03, 46.55it/s]

...

2019-12-17 05:54:36 - Evaluation the model on  dataset after epoch 3:



Epoch:  75%|███████▌  | 3/4 [06:10<01:32, 92.74s/it]

2019-12-17 05:54:40 - Cosine-Similarity :	Pearson: 0.8757	Spearman: 0.8785
2019-12-17 05:54:40 - Manhattan-Distance:	Pearson: 0.8581	Spearman: 0.8636
2019-12-17 05:54:40 - Euclidean-Distance:	Pearson: 0.8592	Spearman: 0.8645
2019-12-17 05:54:40 - Dot-Product-Similarity:	Pearson: 0.8140	Spearman: 0.8265
2019-12-17 05:54:40 - Save model to output/training_stsbenchmark_continue_training-roberta-base-nli-stsb-mean-tokens-2019-12-17_05-48-22
2019-12-17 05:54:40 - Configuration saved in output/training_stsbenchmark_continue_training-roberta-base-nli-stsb-mean-tokens-2019-12-17_05-48-22/0_RoBERTa/config.json


Epoch: 100%|██████████| 4/4 [06:12<00:00, 93.19s/it]

2019-12-17 05:54:42 - Model weights saved in output/training_stsbenchmark_continue_training-roberta-base-nli-stsb-mean-tokens-2019-12-17_05-48-22/0_RoBERTa/pytorch_model.bin


In [8]:
model = SentenceTransformer(model_save_path)
test_data = SentencesDataset(examples=sts_reader.get_examples("sts-dev.csv"), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)
model.evaluate(evaluator)

2019-12-17 05:54:43 - Load pretrained SentenceTransformer: output/training_stsbenchmark_continue_training-roberta-base-nli-stsb-mean-tokens-2019-12-17_05-48-22
2019-12-17 05:54:43 - Load SentenceTransformer from folder: output/training_stsbenchmark_continue_training-roberta-base-nli-stsb-mean-tokens-2019-12-17_05-48-22
2019-12-17 05:54:43 - loading configuration file output/training_stsbenchmark_continue_training-roberta-base-nli-stsb-mean-tokens-2019-12-17_05-48-22/0_RoBERTa/config.json
2019-12-17 05:54:43 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "

Convert Evaluating:   3%|▎         | 5/188 [00:00<00:03, 47.70it/s]

2019-12-17 05:54:46 - Num sentences: 1500
2019-12-17 05:54:46 - Sentences 0 longer than max_seqence_length: 0
2019-12-17 05:54:46 - Sentences 1 longer than max_seqence_length: 0
2019-12-17 05:54:46 - Evaluation the model on  dataset:


Convert Evaluating: 100%|██████████| 188/188 [00:04<00:00, 39.23it/s]

2019-12-17 05:54:51 - Cosine-Similarity :	Pearson: 0.8757	Spearman: 0.8785
2019-12-17 05:54:51 - Manhattan-Distance:	Pearson: 0.8581	Spearman: 0.8636
2019-12-17 05:54:51 - Euclidean-Distance:	Pearson: 0.8592	Spearman: 0.8645
2019-12-17 05:54:51 - Dot-Product-Similarity:	Pearson: 0.8140	Spearman: 0.8265


0.8785484621675564

In [9]:
from sentence_transformers import SentenceTransformer
import scipy.spatial

In [10]:
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr

In [11]:
data3 = open('/home/vv/git/Stsbenchmark/stsbenchmark/sts-dev.csv')
d3 = data3.readlines()
data = d3
print(len(data))

1500


In [12]:
df = []
for i in range(len(data)):
    df.append(data[i].split('\t'))
score = []
sentence1 = []
sentence2 = []
for j in range(len(df)):
    score.append(df[j][4])
    sentence1.append(df[j][5])
    b = df[j][6]
    sentence2.append(b.rstrip("\n"))

In [13]:
dict = list(zip(sentence1, sentence2, score))
df = pd.DataFrame(dict, columns = ['sentence1', 'sentence2', 'score'])

In [14]:
df.head()

,sentence1,sentence2,score
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,5.000
1,A young child is riding a horse.,A child is riding a horse.,4.750
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,5.000
3,A woman is playing the guitar.,A man is playing guitar.,2.400
4,A woman is playing the flute.,A man is playing a flute.,2.750


In [15]:
# scaler = MinMaxScaler()
# df['score'] = scaler.fit_transform(df['score'].values.reshape(-1,1))

In [35]:
df.head(20)

,sentence1,sentence2,score,score_bert
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,5.000,4.984240
1,A young child is riding a horse.,A child is riding a horse.,4.750,4.967064
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,5.000,4.952880
3,A woman is playing the guitar.,A man is playing guitar.,2.400,3.032084
4,A woman is playing the flute.,A man is playing a flute.,2.750,4.105556
5,A woman is cutting an onion.,A man is cutting onions.,2.615,3.161915
6,A man is erasing a chalk board.,The man is erasing the chalk board.,5.000,4.928102
7,A woman is carrying a boy.,A woman is carrying her baby.,2.333,3.577638
8,Three men are playing guitars.,Three men are on stage playing guitars.,3.750,4.149126
9,A woman peels a potato.,A woman is peeling a potato.,5.000,4.959842


In [17]:
closest_n = 1
sentence2=[]
sentence1=[]
score=[]
for i in range(0,len(df)):
    sent1_list = []
    sent2_list=[]
    sent1 = df.sentence1[i]
    sent2 = df.sentence2[i]
    sent1_list.append(sent1)
    sent2_list.append(sent2)
    sent1_embeddings = model.encode(sent1_list)
    sent2_embeddings = model.encode(sent2_list)
    for query, query_embedding in zip(sent2_list, sent2_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sent1_embeddings, "cosine")[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        sentence2.append(sent2_list[0])
        for idx, distance in results[0:closest_n]:
            sentence1.append(sent1_list[0])
            score.append((1-distance)*5)
            print("i : ", i)

Batches: 100%|██████████| 1/1 [00:00<00:00, 95.07it/s]

i :  0
i :  1
i :  2



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

i :  3
i :  4


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

i :  1480
i :  1481
i :  1482
i :  1483
i :  1484
i :  1485
i :  1486
i :  1487
i :  1488


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

i :  1489
i :  1490
i :  1491
i :  1492
i :  1493
i :  1494
i :  1495
i :  1496
i :  1497


Batches: 100%|██████████| 1/1 [00:00<00:00, 91.88it/s]

i :  1498
i :  1499


In [18]:
df['score_bert'] = score
df["score"] = df["score"].apply(pd.to_numeric, downcast='float', errors='coerce')

In [19]:
df.corr()

,score,score_bert
score,1.000000,0.875673
score_bert,0.875673,1.000000


In [31]:
from scipy.stats.stats import pearsonr   
a = df.score
b = df.score_bert  
print (pearsonr(a,b)[0])

0.875673229683211


In [20]:
# scaler = MinMaxScaler()
# df['score_bert'] = scaler.fit_transform(df['score_bert'].values.reshape(-1,1))

In [32]:
s1= ["hi",
     "how are you?",
     "whats up bro how are you?",
     "good night",
     "I LOVE YOU",
     "hey whatsup?",
     "Three men are on stage playing guitars.",
     "A woman is peeling garlic.",
     "phone is not responding.",
     "i love you",
     "How do you book the ticket?",
     "I am going to Afghanisthan?",
     "India defeated west Indies yesterday."]



s2= ["hello",
     "howdy?",
     "Hello bro how are you doing?",
     "good morning",
     "I HATE YOU",
     "hey what you doing?",
     "Three men are playing guitars.",
     "A man is breaking water jugs.",
     "my phone is not working.",
     "I like you",
     "How can the ticket be booked?",
     "I am planning to visit Afghanisthan",
     "West Indies lost to India yesterday."]

In [33]:
closest_n = 1
sentenc2=[]
sentenc1=[]
scor=[]
for i in range(0,len(s2)):
    sent1_list = []
    sent2_list=[]
    sent1 = s1[i]
    sent2 = s2[i]
    sent1_list.append(sent1)
    sent2_list.append(sent2)
    sent1_embeddings = model.encode(sent1_list)
    sent2_embeddings = model.encode(sent2_list)
    for query, query_embedding in zip(sent2_list, sent2_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sent1_embeddings, "cosine")[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        sentenc2.append(sent2_list[0])
        for idx, distance in results[0:closest_n]:
            sentenc1.append(sent1_list[0])
            scor.append(1-distance)

Batches: 100%|██████████| 1/1 [00:00<00:00, 87.12it/s]


In [34]:
for k in range(len(scor)):
    print("sentence 1:", sentenc1[k])
    print("sentence 2:", sentenc2[k])
    print("score:", scor[k])
    print("")

sentence 1: hi
sentence 2: hello
score: 0.9563732486332157

sentence 1: how are you?
sentence 2: howdy?
score: 0.9557301913736441

sentence 1: whats up bro how are you?
sentence 2: Hello bro how are you doing?
score: 0.9766089925130007

sentence 1: good night
sentence 2: good morning
score: 0.9514795816088559

sentence 1: I LOVE YOU
sentence 2: I HATE YOU
score: 0.17871691536618195

sentence 1: hey whatsup?
sentence 2: hey what you doing?
score: 0.9855035207128972

sentence 1: Three men are on stage playing guitars.
sentence 2: Three men are playing guitars.
score: 0.8298251178029261

sentence 1: A woman is peeling garlic.
sentence 2: A man is breaking water jugs.
score: -0.019689163093569606

sentence 1: phone is not responding.
sentence 2: my phone is not working.
score: 0.9522446739628657

sentence 1: i love you
sentence 2: I like you
score: 0.9569670388014893

sentence 1: How do you book the ticket?
sentence 2: How can the ticket be booked?
score: 0.9543446558084833

sentence 1: I 